## Data Reading (CEI DATA) - CG

In [24]:
import pandas as pd
import os

csvs_dir='../data/submission2/cg_comm_counters'

col_names=['version','app','class','cpu','threads','metric','value']

lst_csv_dfs=[]
lst_csv_dfs_sum=[]

for csv_filename in os.listdir(csvs_dir):
    if os.path.isdir(csvs_dir+'/'+csv_filename)==True:
        continue
    lst_csv_dfs.append(pd.read_csv(csvs_dir+'/'+csv_filename, names=col_names))

##apply the core sum preprocessing for each repetition individually    
for df in lst_csv_dfs:
    #drop some rows with multiple header names
    df=df.loc[df['value']!='value'].reset_index(drop=True)
    #casting to numeric
    df.loc[:,'cpu']=pd.to_numeric(df['cpu'])
    df.loc[:,'threads']=pd.to_numeric(df['threads'])
    df.loc[:,'value']=pd.to_numeric(df['value'])
    #summing the value of all cores, for a ['version','app','class','threads','metric'] execution
    df_sum=df.groupby(['version','app','class','threads','metric']).sum().reset_index()
    lst_csv_dfs_sum.append(df_sum)
    
#df_all_cei=pd.concat(lst_csv_dfs)

df_all_cei=pd.concat(lst_csv_dfs_sum)
df_all_cei=df_all_cei.sort_values(['version','app','class','threads','cpu'])

#drop some rows with multiple header names
#df_all_cei=df_all_cei.loc[df_all_cei['value']!='value'].reset_index(drop=True)
#casting to numeric
#df_all_cei.loc[:,'cpu']=pd.to_numeric(df_all_cei['cpu'])
#df_all_cei.loc[:,'threads']=pd.to_numeric(df_all_cei['threads'])
#df_all_cei.loc[:,'value']=pd.to_numeric(df_all_cei['value'])
df_all_cei


,version,app,class,threads,metric,cpu,value
0,native-L1,cg,A,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT,0,0
1,native-L1,cg,A,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM,0,0
2,native-L1,cg,A,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS,0,0
3,native-L1,cg,A,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE,0,840115
0,native-L1,cg,A,1,MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT,0,0
...,...,...,...,...,...,...,...
239,native-none,cg,W,12,MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE,66,46079608
236,native-none,cg,W,12,MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT,66,137904
237,native-none,cg,W,12,MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM,66,9316114
238,native-none,cg,W,12,MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS,66,1176


## Data Preprocessing (CEI DATA)

In [20]:
######## Getting the List of counters ###########################
lst_metrics=df_all_cei.groupby(by='metric').count().reset_index()['metric'].to_list()
lst_metrics
#################################################################

['MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-MISS',
 'MEM-LOAD-L3-HIT-RETIRED-XSNP-NONE']

## L3 DATA

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Javascript,display

#hw_counters=['offcore-response-pf-l2-rfo-l3-hit-any-snoop']
metrics=lst_metrics
nb_threads=['all','1','2','4','8','12']
classes=['W','A','B']

#df_miss_non0=df_miss.loc[df_miss['cpu']==0]
def plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',nb_threads='all',_class='A'):

    #apps=['bt','cg','ep','ft','is','lu','mg','sp','ua']
    apps=['cg']
    
    ##filtering for nas class and metric
    ##the value of the metric is in the column 'value'
    df_metric=df_all_cei.loc[df_all_cei['class']==_class]
    df_metric=df_metric.loc[df_metric['metric']==metric]
    
    if nb_threads!='all':
        df_metric=df_metric.loc[df_metric['threads']==int(nb_threads)]
    #print(df_metric)

    #sns.catplot(x='threads',y='value',hue='version',data=cycles_max_per_threads,kind='bar',col='app',col_wrap=3)
    sns.set(font_scale=1.5)
    #fig, axs=plt.subplots(nrows=3,ncols=3,figsize=(15,15))
    plt.tight_layout(pad=3.5)

    for i,app in enumerate(apps):
        #row=int(i/3)
        #col=int(i%3)
        cycles_app=df_metric.loc[df_metric['app']==app]
        #print(app)        
        if nb_threads=='all':
            ax=sns.barplot(x='threads',y='value',hue='version',data=cycles_app,
                        hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            # legend title
            ax.legend(framealpha=0.4)
            new_title = 'Execution'
            ax.legend_.set_title(new_title)            
            # replace labels
            new_labels = ['Real-L1-L2', 'Real-L2', 'Real-L1', 'Real-NoPref']
            for t, l in zip(ax.legend_.texts, new_labels): t.set_text(l)
            #sns.barplot(x='threads',y=metric,hue='version',data=cycles_app,ax=axs[row][col], hue_order=['native-L1-L2','native-L2','native-L1','native-none','zsim','sniper-none','sniper-l1','sniper-l1-l2'])
        else:
            sns.barplot(x='cpu',y='value',hue='version',data=cycles_app,hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            ax.legend_.remove()
        
        #sns.barplot(x='threads',y='value',hue='version',data=cycles_app,ax=axs[row][col])
        #sns.catplot(x='threads',y='value',hue='version',data=cycles_app,kind='bar',ax=axs[row][col])
        ax.set_title(app.upper()+' (class '+_class+')')
        ax.set_ylabel(metric)
        if nb_threads=='all':
            ax.set_xlabel('Number of Threads')
        else:
            ax.set_xlabel('Core Index')
        #axs[row][col].set_yscale('log')
        #break

    plt.savefig('../figures/submission2/CG_'+metric+'_'+_class+'_'+nb_threads+'_subm2_cei.pdf',type='pdf',bbox_inches='tight')
    
dropdown_metrics=widgets.Dropdown(
    options=metrics,
    value='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM',
    description='metric',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)   
dropdown_classes=widgets.Dropdown(
    options=classes,
    value='A',
    description='class',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
dropdown_nb_threads=widgets.Dropdown(
    options=nb_threads,
    value='all',
    description='nb_threads',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact_manual(plot_metrics_graph,                
                metric=dropdown_metrics, 
                nb_threads=dropdown_nb_threads, 
                _class=dropdown_classes)

interactive(children=(Dropdown(description='metric', index=1, options=('MEM-LOAD-L3-HIT-RETIRED-XSNP-HIT', 'ME…

<function __main__.plot_metrics_graph(metric='MEM-LOAD-L3-HIT-RETIRED-XSNP-HITM', nb_threads='all', _class='A')>

## Reading standard data to plot mem access plot

In [34]:
import pandas as pd
import os

csvs_dir='../data/submission2/native/nas/cei'

col_names=['version','app','class','cpu','threads','metric','value']
lst_metrics=['PAPI-TOT-INS', 
             'PAPI-TOT-CYC',             
             'L2-RQSTS-DEMAND-DATA-RD-HIT', 
             'L2-RQSTS-DEMAND-DATA-RD-MISS', 
             'L2-RQSTS-ALL-PF', 
             'L2-RQSTS-PF-HIT', 
             'L2-RQSTS-PF-MISS', 
             'L2-LINES-OUT-USELESS-HWPREF', 
             'L2-LINES-OUT-USELESS-HWPF', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-ANY-RESPONSE', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-L3-HIT-SNP-ANY', 
             'OFFCORE-RESPONSE-0-PF-L2-DATA-RD-L3-HIT-SNP-HITM', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-ANY-RESPONSE', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-L3-HIT-SNP-ANY', 
             'OFFCORE-RESPONSE-0-PF-L2-RFO-L3-HIT-SNP-HITM', 
             'OFFCORE-REQUESTS-DEMAND-DATA-RD', 
             'OFFCORE-REQUESTS-ALL-DATA-RD', 
             'OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD']

lst_csv_dfs=[]
lst_csv_dfs_sum=[]

for csv_filename in os.listdir(csvs_dir):
    if os.path.isdir(csvs_dir+'/'+csv_filename)==True:
        continue
    lst_csv_dfs.append(pd.read_csv(csvs_dir+'/'+csv_filename, names=col_names))
    
##apply the core sum preprocessing for each repetition individually    
for df in lst_csv_dfs:
    #drop some rows with multiple header names
    #df=df.loc[df['value']!='value'].reset_index(drop=True)
    #casting to numeric
    #df.loc[:,'cpu']=pd.to_numeric(df['cpu'])
    #df.loc[:,'threads']=pd.to_numeric(df['threads'])
    #df.loc[:,'value']=pd.to_numeric(df['value'])
    #summing the value of all cores, for a ['version','app','class','threads','metric'] execution
    df_sum=df.groupby(['version','app','class','threads','metric']).sum().reset_index()
    lst_csv_dfs_sum.append(df_sum)

#df_all=pd.concat(lst_csv_dfs)
df_all=pd.concat(lst_csv_dfs_sum)
df_all=df_all.sort_values(['version','app','class','threads','cpu'])
 
##dropping the PAPI-TOT-CYC and PAPI-TOT-INS because each csv file has two repetitions
## of the execution of these two metrics, leading to a wrong sum value
df_all=df_all.loc[df_all['metric']!='PAPI-TOT-INS']
df_all=df_all.loc[df_all['metric']!='PAPI-TOT-CYC']
##printing 'OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD' because it's the one we want in the paper
df_all.loc[df_all['metric']=='OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD']

,version,app,class,threads,metric,cpu,value
9,native-L1,bt,A,1,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,0,58952339
9,native-L1,bt,A,1,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,0,59167668
9,native-L1,bt,A,1,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,0,60022932
9,native-L1,bt,A,1,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,0,60177014
9,native-L1,bt,A,1,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,0,60455941
...,...,...,...,...,...,...,...
2421,native-none,ua,W,12,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,66,219177
2421,native-none,ua,W,12,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,66,60759
2421,native-none,ua,W,12,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,66,91934
2421,native-none,ua,W,12,OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD,66,219191


In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Javascript,display

#hw_counters=['offcore-response-pf-l2-rfo-l3-hit-any-snoop']
metrics=lst_metrics
nb_threads=['all','1','2','4','8','12']
classes=['W','A','B']

#df_miss_non0=df_miss.loc[df_miss['cpu']==0]
def plot_metrics_graph(metric='OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD',nb_threads='all',_class='A'):

    #apps=['bt','cg','ep','ft','is','lu','mg','sp','ua']
    apps=['cg']
    
    ##filtering for nas class and metric
    ##the value of the metric is in the column 'value'
    df_metric=df_all.loc[df_all['class']==_class]
    df_metric=df_metric.loc[df_metric['metric']==metric]
    
    if nb_threads!='all':
        df_metric=df_metric.loc[df_metric['threads']==int(nb_threads)]
    #print(df_metric)

    #sns.catplot(x='threads',y='value',hue='version',data=cycles_max_per_threads,kind='bar',col='app',col_wrap=3)
    sns.set(font_scale=1.5)
    #fig, axs=plt.subplots(nrows=3,ncols=3,figsize=(15,15))
    plt.tight_layout(pad=3.5)

    for i,app in enumerate(apps):
        #row=int(i/3)
        #col=int(i%3)
        cycles_app=df_metric.loc[df_metric['app']==app]
        #print(app)        
        if nb_threads=='all':
            ax=sns.barplot(x='threads',y='value',hue='version',data=cycles_app,
                        hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            # legend title
            ax.legend(framealpha=0.4)
            new_title = 'Execution'
            ax.legend_.set_title(new_title)            
            # replace labels
            new_labels = ['Real-L1-L2', 'Real-L2', 'Real-L1', 'Real-NoPref']
            for t, l in zip(ax.legend_.texts, new_labels): t.set_text(l)
            #sns.barplot(x='threads',y=metric,hue='version',data=cycles_app,ax=axs[row][col], hue_order=['native-L1-L2','native-L2','native-L1','native-none','zsim','sniper-none','sniper-l1','sniper-l1-l2'])
        else:
            sns.barplot(x='cpu',y='value',hue='version',data=cycles_app,hue_order=['native-L1-L2','native-L2','native-L1','native-none'],palette='cubehelix')
            ax.legend_.remove()
        
        #sns.barplot(x='threads',y='value',hue='version',data=cycles_app,ax=axs[row][col])
        #sns.catplot(x='threads',y='value',hue='version',data=cycles_app,kind='bar',ax=axs[row][col])
        ax.set_title(app.upper()+' (class '+_class+')')
        ax.set_ylabel(metric)
        if nb_threads=='all':
            ax.set_xlabel('Number of Threads')
        else:
            ax.set_xlabel('Core Index')
        ax.set_yscale('log')
        #break

    plt.savefig('../figures/submission2/CG_'+metric+'_'+_class+'_'+nb_threads+'_subm2_cei.pdf',type='pdf',bbox_inches='tight')
    
dropdown_metrics=widgets.Dropdown(
    options=metrics,
    value='OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD',
    description='metric',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)   
dropdown_classes=widgets.Dropdown(
    options=classes,
    value='A',
    description='class',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
dropdown_nb_threads=widgets.Dropdown(
    options=nb_threads,
    value='all',
    description='nb_threads',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

interact_manual(plot_metrics_graph,                
                metric=dropdown_metrics, 
                nb_threads=dropdown_nb_threads, 
                _class=dropdown_classes)

interactive(children=(Dropdown(description='metric', index=17, options=('PAPI-TOT-INS', 'PAPI-TOT-CYC', 'L2-RQ…

<function __main__.plot_metrics_graph(metric='OFFCORE-REQUESTS-L3-MISS-DEMAND-DATA-RD', nb_threads='all', _class='A')>